In [1]:
from imblearn.over_sampling import RandomOverSampler,SMOTE,BorderlineSMOTE
import tensorflow as tf
import tensorflow.keras.layers as layers
import pandas as pd
import numpy as np
import os
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import model_selection

In [2]:
features = 8
num_classes = 9
latent_dim = 4
shape = (8,1)

In [3]:
##数据处理部分
#用train_data  做训练集， 用blind_data盲井做验证集
filename = './Dataset/美国油田/all_data不含PE缺失.xlsx'
data=pd.read_excel(filename)
data['Formation'] = pd.Categorical(data['Formation'])
data['Formation'] = data.Formation.cat.codes
train_data = data[~data['Well Name'].str.contains('STUART|CRAWFORD')]
test_data = data[data['Well Name'].str.contains('STUART|CRAWFORD')]
data_x = train_data.drop(['Well Name','Depth','Facies'],axis=1)
data_x = data_x.values
data_y = train_data['Facies']-1
y_train = data_y.values
# y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
#标准化
max_min = preprocessing.StandardScaler()
X_train = max_min.fit_transform(data_x)
# X_train = np.expand_dims(data_x,2)
#验证集
X_eval = test_data.drop(['Well Name','Depth','Facies'],axis=1)
y_eval = test_data['Facies']-1
y_eval = y_eval.values
# y_eval = tf.keras.utils.to_categorical(y_eval, num_classes=num_classes)
#标准化
X_eval = X_eval.values
X_eval = max_min.fit_transform(X_eval)
X_eval = np.expand_dims(X_eval,2)


In [24]:
#数据分割 80%
X_train_,X_test_,y_train_,y_test_ = model_selection.train_test_split(X_train,y_train,test_size=0.2,stratify=y_train)

In [40]:
#数据分割 60%
X_train_,X_test_,y_train_,y_test_ = model_selection.train_test_split(X_train,y_train,test_size=0.4,stratify=y_train)

In [4]:
ros = RandomOverSampler(random_state=123456)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
X_resampled.shape , y_resampled.shape

((6642, 8), (6642,))

In [52]:
ros = RandomOverSampler(random_state=123456)
X_resampled, y_resampled = ros.fit_resample(X_train_, y_train_)
X_resampled.shape , y_resampled.shape

((3987, 8), (3987,))

In [ ]:
X_resampled,y_resampled = SMOTE().fit_resample(X_train_, y_train_)
X_resampled.shape , y_resampled.shape

In [32]:
X_resampled,y_resampled = BorderlineSMOTE().fit_resample(X_train_, y_train_)
X_resampled.shape , y_resampled.shape

((5310, 8), (5310,))

In [5]:
def Classifier():
  model = tf.keras.Sequential()
  model.add(layers.Conv1D(filters = 256,kernel_size = 3, padding = 'same', activation='relu',input_shape=shape))
  model.add(layers.Dropout(0.5))
  model.add(layers.MaxPooling1D(2,2))
  model.add(layers.Conv1D(filters = 128,kernel_size = 3, padding = 'same', activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.MaxPooling1D(2,2))
  model.add(layers.Conv1D(filters = 64,kernel_size = 3, padding = 'same', activation='relu'))
  model.add(layers.MaxPooling1D(2,2))
  model.add(layers.Flatten())
  model.add(layers.Dense(1500,activation='relu'))
  model.add(layers.Dense(500,activation='relu'))
  model.add(layers.Dense(100,activation='relu'))
  model.add(layers.Dense(num_classes,activation='softmax'))
  model.summary()
  return model

In [ ]:
for i in range(10):
    X_train_,X_test_,y_train_,y_test_ = model_selection.train_test_split(X_train,y_train,test_size=0.8,stratify=y_train)
    X_resampled, y_resampled = ros.fit_resample(X_train_, y_train_)
    X_resampled = np.expand_dims(X_resampled,2)
    model = Classifier()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
    history = model.fit(X_resampled,y_resampled,epochs=100,batch_size=256,validation_data=(X_eval, y_eval),verbose=0)
    print(max(history.history['val_accuracy']))

In [9]:
ros = RandomOverSampler(random_state=123456)
smo = SMOTE(random_state=123456)

In [ ]:
for i in range(10):
    X_train_,X_test_,y_train_,y_test_ = model_selection.train_test_split(X_train,y_train,test_size=0.8,stratify=y_train)
    X_resampled, y_resampled = smo.fit_resample(X_train_, y_train_)
    X_resampled = np.expand_dims(X_resampled,2)
    model = Classifier()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
    history = model.fit(X_resampled,y_resampled,epochs=100,batch_size=256,validation_data=(X_eval, y_eval),verbose=0)
    print(max(history.history['val_accuracy']))

In [ ]:
X_resampled = np.expand_dims(X_resampled,2)
model = Classifier()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
history = model.fit(X_resampled,y_resampled,epochs=100,batch_size=256,validation_data=(X_eval, y_eval))

In [2]:
ros = RandomOverSampler(random_state=123456)
smo = SMOTE(random_state=123456)

In [3]:
filename = 'E:/数据集/DGF-HGF.xlsx'
DGF_data = pd.read_excel(filename,sheet_name='DGF data')
HGF_data = pd.read_excel(filename,sheet_name='HGF data')
dist_DGF = {'FS':0, 'MS':1, 'CS':2, 'PS':3, 'CR':4, 'S':5, 'M':6, 'C':7}
dist = {'FS':0, 'MS':1, 'CS':2, 'PS':3, 'CR':4, 'S':5, 'M':6, 'C':7}
all_data = pd.concat([DGF_data,HGF_data])
all_data['Face'] = all_data['Type'].map(dist)
x_data = pd.DataFrame(all_data,columns=['GR','AC','DEN','CNL','LLD','LLS','CAL'])
y_data = pd.DataFrame(all_data,columns=['Face'])
x_data = x_data.values
y_data = y_data.values
max_min = preprocessing.StandardScaler()
x_data = max_min.fit_transform(x_data)
all_data_ = np.concatenate((x_data,y_data.reshape(-1,1)),axis=1)
np.random.shuffle(all_data_)
x_data_ = all_data_[:,:-1]
y_data_ = all_data_[:,-1]
x_data_ = np.expand_dims(x_data_,2)
# y_data_ = np.expand_dims(y_data_,axis=1)

In [8]:
#8 2 分割训练和验证 
train_x,val_x,train_y,val_y=model_selection.train_test_split(x_data_,y_data_,test_size=0.2,stratify=y_data_)

In [5]:
def Classifier():
  model = tf.keras.Sequential()
  model.add(layers.Conv1D(filters = 64,kernel_size = 3, padding = 'same',input_shape=(7,1)))  
  model.add(layers.BatchNormalization())
  model.add(layers.Activation('relu'))
  model.add(layers.MaxPooling1D(2,1))
  model.add(layers.Conv1D(filters = 128,kernel_size = 3, padding = 'same'))  
  model.add(layers.BatchNormalization())
  model.add(layers.Activation('relu'))
  model.add(layers.MaxPooling1D(2,1))
  model.add(layers.Conv1D(filters = 128,kernel_size = 3, padding = 'same'))
  model.add(layers.BatchNormalization())
  model.add(layers.Activation('relu'))
  model.add(layers.MaxPooling1D(2,1))
  model.add(layers.Flatten())
  model.add(layers.Dense(1500))
  model.add(layers.BatchNormalization())
  model.add(layers.Activation('relu'))
  model.add(layers.Dense(500))
  model.add(layers.BatchNormalization())
  model.add(layers.Activation('relu'))
  model.add(layers.Dense(8,activation='softmax'))
  return model

In [10]:
train_x  = train_x.reshape(-1,7)
train_x.shape

(1722, 7)

In [ ]:
for i in range(10):
    X_resampled, y_resampled = ros.fit_resample(train_x, train_y)
    X_resampled = np.expand_dims(X_resampled,2)
    model = Classifier()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
    history = model.fit(X_resampled,y_resampled,epochs=200,batch_size=256,validation_data=(val_x, val_y),verbose=0)
    print(max(history.history['val_accuracy']))

In [ ]:
for i in range(10):
    X_resampled, y_resampled = smo.fit_resample(train_x, train_y)
    X_resampled = np.expand_dims(X_resampled,2)
    model = Classifier()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
    history = model.fit(X_resampled,y_resampled,epochs=200,batch_size=256,validation_data=(val_x, val_y),verbose=0)
    print(max(history.history['val_accuracy']))

In [ ]:
for i in range(10):
    X_train_,X_test_,y_train_,y_test_ = model_selection.train_test_split(train_x,train_y,test_size=0.2,stratify=train_y)
    X_resampled, y_resampled = ros.fit_resample(X_train_, y_train_)
    X_resampled = np.expand_dims(X_resampled,2)
    model = Classifier()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
    history = model.fit(X_resampled,y_resampled,epochs=200,batch_size=256,validation_data=(val_x, val_y),verbose=0)
    print(max(history.history['val_accuracy']))

In [ ]:
for i in range(10):
    X_train_,X_test_,y_train_,y_test_ = model_selection.train_test_split(train_x,train_y,test_size=0.2,stratify=train_y)
    X_resampled, y_resampled = smo.fit_resample(X_train_, y_train_)
    X_resampled = np.expand_dims(X_resampled,2)
    model = Classifier()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
    history = model.fit(X_resampled,y_resampled,epochs=200,batch_size=256,validation_data=(val_x, val_y),verbose=0)
    print(max(history.history['val_accuracy']))

In [ ]:
for i in range(10):
    X_train_,X_test_,y_train_,y_test_ = model_selection.train_test_split(train_x,train_y,test_size=0.4,stratify=train_y)
    X_resampled, y_resampled = ros.fit_resample(X_train_, y_train_)
    X_resampled = np.expand_dims(X_resampled,2)
    model = Classifier()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
    history = model.fit(X_resampled,y_resampled,epochs=200,batch_size=256,validation_data=(val_x, val_y),verbose=0)
    print(max(history.history['val_accuracy']))

In [ ]:
for i in range(10):
    X_train_,X_test_,y_train_,y_test_ = model_selection.train_test_split(train_x,train_y,test_size=0.4,stratify=train_y)
    X_resampled, y_resampled = smo.fit_resample(X_train_, y_train_)
    X_resampled = np.expand_dims(X_resampled,2)
    model = Classifier()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
    history = model.fit(X_resampled,y_resampled,epochs=200,batch_size=256,validation_data=(val_x, val_y),verbose=0)
    print(max(history.history['val_accuracy']))

In [ ]:
for i in range(10):
    X_train_,X_test_,y_train_,y_test_ = model_selection.train_test_split(train_x,train_y,test_size=0.6,stratify=train_y)
    X_resampled, y_resampled = ros.fit_resample(X_train_, y_train_)
    X_resampled = np.expand_dims(X_resampled,2)
    model = Classifier()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
    history = model.fit(X_resampled,y_resampled,epochs=200,batch_size=256,validation_data=(val_x, val_y),verbose=0)
    print(max(history.history['val_accuracy']))

In [ ]:
for i in range(10):
    X_train_,X_test_,y_train_,y_test_ = model_selection.train_test_split(train_x,train_y,test_size=0.6,stratify=train_y)
    X_resampled, y_resampled = smo.fit_resample(X_train_, y_train_)
    X_resampled = np.expand_dims(X_resampled,2)
    model = Classifier()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
    history = model.fit(X_resampled,y_resampled,epochs=200,batch_size=256,validation_data=(val_x, val_y),verbose=0)
    print(max(history.history['val_accuracy']))